In [10]:
import rpy2
import os

base_directory = "C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning/"
r_directory = "C:/PROGRA~1/R/R-44~1.2"
os.environ['R_HOME'] = r_directory
import rpy2.robjects as robjects
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
from rpy2.robjects.packages import importr,data
utils = importr('utils')
base = importr('base')
from pynm.pynm import PyNM
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import patsy as pat
import statsmodels.api as sm
from distfit import distfit
import re
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri
from rpy2.robjects import pandas2ri
from rpy2.robjects import r
from pynm.util import read_confounds

In [11]:
gamlss_data = importr('gamlss.data')
gamlss_dist = importr('gamlss.dist')
gamlss = importr('gamlss')

In [12]:
df_combined_long = pd.read_csv(base_directory + "df_combined_long.csv")
df_combined_long['aircon_type_simplified'] = df_combined_long['aircon_type_simplified'].astype('category')
df_combined_long['property_construction'] = df_combined_long['property_construction'].astype('category')  
df_combined_long['climate_zone'] = df_combined_long['climate_zone'].astype('category')  


dist = distfit()        # Initialize 
# remove NA values from the daily_kWh_consumption column
df_combined_long = df_combined_long.dropna(subset=['daily_kWh_consumption'])
# remove rows where daily_kWh_consumption is less than 1
df_combined_long = df_combined_long[df_combined_long['daily_kWh_consumption'] >= 1]
# dist.fit_transform(df_combined_long['daily_kWh_consumption'].values)   # Fit distributions on empirical data X
# dist.predict(df_combined_long['daily_kWh_consumption'].values)         # Predict the probability of the resonse variables
# dist.plot()


In [13]:
m = PyNM(df_combined_long, 'daily_kWh_consumption', confounds = ['aircon_type_simplified', 'c(property_construction)', 'c(climate_zone)'])   

TypeError: PyNM.__init__() missing 1 required positional argument: 'group'

In [14]:
# GAMLSS function
class GAMLSS:
    """Class for GAMLSS model.
        
        Attributes
        ----------
        gamlss_data: R package
            Python imported R package.
        gamlss_dist: R package
            Python imported R package.
        gamlss: R package
            Python imported R package.
        mu_f: R formula
            Formula for mu (location) parameter.
        sigma_f: R formula
            Formula for sigma (scale) parameter.
        nu_f: R formula
            Formula for nu (skewness) parameter.
        tau_f: R formula
            Formula for tau (kurtosis) parameter.
        rfamily: R object
            Family of distributions to use for fitting.
        method: str
            Method to fit GAMLSS.
        model: R object
            Fitted GAMLSS model.
        """

    def __init__(self,mu=None,sigma=None,nu=None,tau=None,family='SHASHo2',method='RS',score=None,confounds=None):
        """Create GAMLSS object. Formulas must be written for R, using functions available in the GAMLSS package.
        
        Parameters
        ----------
        mu: str, default=None
            Formula for mu (location) parameter of GAMLSS. If None, formula for score is sum of confounds
            with non-categorical columns as smooth functions, e.g. "score ~ ps(age) + sex".
        sigma: str, default=None
            Formula for sigma (scale) parameter of GAMLSS. If None, formula is '~ 1'.
        nu: str, default=None
            Formula for nu (skewness) parameter of GAMLSS. If None, formula is '~ 1'.
        tau: str, default=None
            Formula for tau (kurtosis) parameter of GAMLSS. If None, formula is '~ 1'.
        family: str,default='SHASHo2'
            Family of distributions to use for fitting, default is 'SHASHo2'. See R documentation for GAMLSS package for other available families of distributions.
        method: str, default = 'RS'
            Method for fitting GAMLSS. Can be 'RS' (Rigby and Stasinopoulos algorithm), 'CG' (Cole and Green algorithm) or 'mixed(n,m)' where n & m are integers.
            Specifying 'mixed(n,m)' will use the RS algorithm for n iterations and the CG algorithm for up to m additional iterations.
        score: str, default=None
            Label of score in DataFrame.
        confounds: list, default=None
            List of labels of confounds in DataFrame.
        
        Notes
        -----
        If using 'random()' to model a random effect in any of the formulas, it must be passed a column of the dataframe with categorical values
        as a factor: e.g. 'random(as.factor(COL))'.
        """
        numpy2ri.activate()
        pandas2ri.activate()

        self.gamlss_data = importr('gamlss.data')
        self.gamlss_dist = importr('gamlss.dist')
        self.gamlss = importr('gamlss')
        self.base = importr('base')
        
        self.score = score
        self.confounds = confounds
        self.mu_f,self.sigma_f,self.nu_f,self.tau_f = self._get_r_formulas(mu,sigma,nu,tau)
        self.family = family
        self.method = self._get_method(method)
        try:
            self.rfamily = r[family]
        except:
            raise ValueError("Provided family not valid, choose 'SHASHo2', 'NO' or see R documentation for GAMLSS package for other available families of distributions.")
    
    def _get_r_formulas(self,mu,sigma,nu,tau):
        """Convert from string input to R formula.

        Parameters
        ----------
        mu: str or None
            Formula for mu (location) parameter of GAMLSS. If None, formula for score is sum of confounds
            with non-categorical columns as smooth functions, e.g. "score ~ ps(age) + sex".
        sigma: str or None
            Formula for sigma (scale) parameter of GAMLSS. If None, formula is '~ 1'.
        nu: str or None
            Formula for nu (skewness) parameter of GAMLSS. If None, formula is '~ 1'.
        tau: str or None
            Formula for tau (kurtosis) parameter of GAMLSS. If None, formula is '~ 1'.

        Raises
        ------
        ValueError
            If any of the input strings contains a function call not recognised by the R GAMLSS package.
        ValueError
            If mu is None and either score or confounds is None.
        
        Returns
        -------
        R formula, R formula, R formula, R formula
            R formula equivalent for each input string.
        """
        if mu is None:
            if (self.score is None) or (self.confounds is None):
                raise ValueError('If mu is None, both score and confounds must be provided i.e. not None.')
            _,cat = read_confounds(self.confounds)
            formula_conf = ['ps({})'.format(conf) for conf in self.confounds if not conf[2:-1] in cat] + cat
            mu = '{} ~ {}'.format(self.score,' + '.join(formula_conf))
        if sigma is None:
            sigma = '~ 1'
        if nu is None:
            nu = '~ 1'
        if tau is None:
            tau = '~ 1'
        
        # get r functions from formulas
        p = re.compile(r"\w*\(")
        funcs = []
        for s in [mu,sigma,nu,tau]:
            for f in p.findall(s):
                funcs.append(f[:-1])

        for func in funcs:
            try:
                exec("{} = r['{}']".format(func,func))
            except:
                raise ValueError("'{}' function not found in R GAMLSS package. See GAMLSS documentation for available functions.".format(func))

        return mu,sigma,nu,tau
    
    def _get_method(self,method):
        """ Get method parameter in appropriate format for R.

        Raises
        ------
        TypeError
            "Argument 'method' must be of type str."
        ValueError
            "Unrecognized argument for 'method'."
        """
        if not isinstance(method,str):
            raise TypeError("Argument 'method' must be of type str.")

        pattern = re.compile(r"mixed\([0-9]*,[0-9]*\)")
    
        if method == 'RS':
            return 'RS()'
        elif method == 'CG':
            return 'CG()'
        elif pattern.match(method) is not None:
            return method
        else:
            raise ValueError("Unrecognized argument for 'method'.")
    
    def fit(self,train_data):
        """Create and fit gamlss model.

        Parameters
        ----------
        train_data: DataFrame
            DataFrame with training data.
        """
        ro.globalenv['train_data'] = train_data

        self.model = r(f'''gamlss({self.mu_f},
                                sigma.formula={self.sigma_f},
                                nu.formula={self.nu_f},
                                tau.formula={self.tau_f},
                                family={self.family},
                                data=train_data,
                                method={self.method})''')
                    
    def predict(self,test_data,what='mu'):
        """Predict from fitted gamlss model.
        
        Parameters
        ----------
        test_data: DataFrame
            DataFrame with test data.
        what: str
            Which parameter to predict, can be 'mu','sigma', 'nu', or 'tau'.
        """
        ro.globalenv['model'] = self.model
        ro.globalenv['test_data'] = test_data

        res = r(f'''predict(model,newdata=test_data,parameter="{what}")''')
        return res

In [27]:
prep = df_combined_long[['daily_kWh_consumption', 'aircon_type_simplified', 'property_construction', 'climate_zone', 'num_occupants']]
prep['climate_zone'] = prep['climate_zone'].astype('category')

In [ ]:
from sklearn.model_selection import train_test_split

X = prep.drop(columns=['daily_kWh_consumption'])
y = prep['daily_kWh_consumption']

X_train, X_test, y_train, y_test = train_test_split(X,y , 
                                   random_state=104,  
                                   test_size=1,  
                                   shuffle=True) 

def prepare_gamlss_data(X, y, target_name='daily_consumption'):
    """
    Combine features and target into a single DataFrame for GAMLSS with NA handling.
    """
    # Create combined dataset
    data = X.copy()
    data[target_name] = y
    
    # Handle missing values
    # 1. For numeric columns: fill with median
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        data[col] = data[col].fillna(data[col].median())
    
    # 2. For categorical columns: fill with mode
    categorical_cols = data.select_dtypes(exclude=[np.number]).columns
    for col in categorical_cols:
        data[col] = data[col].fillna(data[col].mode()[0])
    
    # 3. Verify no NAs remain
    if data.isna().any().any():
        raise ValueError("Data still contains NA values after preprocessing")
        
    return data

train_data = prepare_gamlss_data(X_train, y_train)
test_data = prepare_gamlss_data(X_test, y_test)

print(train_data.head())

             aircon_type_simplified property_construction climate_zone  \
71242                        Ducted          Double brick            6   
65739                        Ducted            Don't know            5   
23943                        Ducted          Brick veneer            6   
68185                        Ducted          Double brick            5   
36403  Wall / window mounted aircon           Lightweight            6   

       num_occupants  daily_consumption  
71242              4           7.277271  
65739              4           6.761757  
23943              4           1.816815  
68185              4           4.139234  
36403              2           3.081395  


In [43]:
import pandas as pd
from rpy2.robjects import pandas2ri, r
import rpy2.robjects as ro
from rpy2.robjects.packages import importr

# Initialize GAMLSS model for daily consumption
gamlss_model = GAMLSS(
    # Formula for mu (location parameter) - mean consumption
    mu="daily_consumption ~ aircon_type_simplified + property_construction + num_occupants + climate_zone",
    
    # Formula for sigma (scale parameter) - variance structure
    # Allowing variance to vary by property type and number of occupants
    sigma="~ aircon_type_simplified + property_construction + num_occupants",
    
    # Using default SHASHo2 distribution which allows for flexible modeling
    family='SHASHo2'
)

# Assuming you have your data in a pandas DataFrame called 'data'
# Fit the model
gamlss_model.fit(train_data)

# Make predictions for both mean (mu) and variance (sigma)
# For mean predictions
#mu_predictions = gamlss_model.predict(test_data, what='mu')

# For variance predictions
#sigma_predictions = gamlss_model.predict(test_data, what='sigma')

GAMLSS-RS iteration 1: Global Deviance = 34538.32 
GAMLSS-RS iteration 2: Global Deviance = 33889.18 
GAMLSS-RS iteration 3: Global Deviance = 33466.84 
GAMLSS-RS iteration 4: Global Deviance = 33153 
GAMLSS-RS iteration 5: Global Deviance = 32981.27 
GAMLSS-RS iteration 6: Global Deviance = 32876.42 
GAMLSS-RS iteration 7: Global Deviance = 32809.42 
GAMLSS-RS iteration 8: Global Deviance = 32765.91 
GAMLSS-RS iteration 9: Global Deviance = 32735.77 
GAMLSS-RS iteration 10: Global Deviance = 32714.4 
GAMLSS-RS iteration 11: Global Deviance = 32698.21 
GAMLSS-RS iteration 12: Global Deviance = 32685.57 
GAMLSS-RS iteration 13: Global Deviance = 32675.46 
GAMLSS-RS iteration 14: Global Deviance = 32667.03 
GAMLSS-RS iteration 15: Global Deviance = 32660.05 
GAMLSS-RS iteration 16: Global Deviance = 32654 
GAMLSS-RS iteration 17: Global Deviance = 32648.72 
GAMLSS-RS iteration 18: Global Deviance = 32644.07 
GAMLSS-RS iteration 19: Global Deviance = 32639.93 
GAMLSS-RS iteration 20: Glob

[distfit] >WARNING> R[write to console]: In addition: 
[distfit] >WARNING> R[write to console]: Warning message:

[distfit] >WARNING> R[write to console]: In RS() :
[distfit] >WARNING> R[write to console]:  Algorithm RS has not yet converged



In [39]:
def print_gamlss_summary(gamlss_model):
    """
    Print the original R summary of the GAMLSS model
    """
    try:
        # Assign model to R environment
        ro.globalenv['model'] = gamlss_model.model
        
        # Print full summary
        ro.r('summary(model)')
        
    except Exception as e:
        print(f"Error printing summary: {str(e)}")

# Print the summary
print_gamlss_summary(gamlss_model)

******************************************************************
Family:  c("SHASHo2", "Sinh-Arcsinh") 

Call:  gamlss(formula = daily_consumption ~ aircon_type_simplified +  
    property_construction + num_occupants + climate_zone,  
    sigma.formula = ~aircon_type_simplified + property_construction +  
        num_occupants, nu.formula = ~1, tau.formula = ~1,  
    family = SHASHo2, data = train_data, method = RS()) 

Fitting method: RS() 

------------------------------------------------------------------
Mu link function:  identity
Mu Coefficients:
                                                    Estimate Std. Error t value
(Intercept)                                         0.979447   0.146248   6.697
aircon_type_simplifiedSplit System                 -0.089438   0.027080  -3.303
aircon_type_simplifiedWall / window mounted aircon  0.248385   0.135145   1.838
property_constructionConcrete                       0.176854   0.036435   4.854
property_constructionDon't know      